In [1]:
# !pip install thefuzz[speedup]


In [2]:
import os
import pandas as pd
from thefuzz import process
import re

In [3]:

input_folder = r"C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Preparation\\dataset\\final_data"
output_path = r"C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Preparation\\document_data\\documents(new).csv"

In [3]:
reference_file = r"C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Scraper\\Notebook\\dest_titles.csv"

In [ ]:
def clean_weird_chars(text):
    if pd.isna(text):
        return text
    return (text.replace("â", "’")
                .replace("â", "–")
                .replace("Â", "’")
                .replace("Ã¢ÂÂ", "’")
                .replace("Ã¢ÂÂ", "–"))


df1 = pd.read_csv(reference_file, encoding="utf-8-sig")
df1["Destination"] = df1["Destination"].apply(clean_weird_chars)
correct_titles = df1["Destination"].tolist()

unwanted_keywords = ["Unclaim", "Someone from this business", "Claim this listing", "If you own this business"]

for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path, encoding="utf-8-sig")
        
        if "Destination Title" in df.columns:
            for i, title in enumerate(df["Destination Title"]):
                if pd.isna(title):
                    continue
                
                # Remove unwanted keywords
                cleaned_title = str(title)
                for kw in unwanted_keywords:
                    cleaned_title = cleaned_title.split(kw)[0].strip()
                
                # Clean before fuzzy match
                cleaned_title = clean_weird_chars(cleaned_title)
                
                # Fuzzy match to reference
                best_match, score = process.extractOne(cleaned_title, correct_titles)
                if score >= 80:
                    # Save the cleaned match
                    df.at[i, "Destination Title"] = clean_weird_chars(best_match)
                
            # Save CSV with proper UTF-8 encoding
            df.to_csv(file_path, index=False, encoding="utf-8-sig")
            print(f"Updated {file_name}")
        else:
            print(f"Skipped {file_name}: 'Destination Title' column not found")


Updated Abayagiriya Stupa.csv
Updated Aberdeen Falls.csv
Updated Alagalla Mountain Range.csv
Updated Aluvihara Rock Cave Temple.csv
Updated Ambuluwawa Tower.csv
Updated Anawilundawa Wetland Sanctuary.csv
Updated Ancient City of Polonnaruwa.csv
Updated Arcade Independence Square.csv
Updated Archaeological Museum.csv
Updated Arugam Bay.csv
Updated Avukana Buddha Statue.csv
Updated Baker's Falls.csv
Updated Bambarakanda Falls.csv
Updated Bandaranaike Memorial International Conference Hall.csv
Updated Baobab Tree.csv
Updated Beira Lake.csv
Updated Bentota Beach.csv
Updated Beruwala Beach.csv
Updated Bluefield Tea Gardens.csv
Updated Bomburu Ella Waterfall.csv
Updated Bopath Ella (Bopath Falls).csv
Updated Buduruwagala Temple.csv
Updated Bundala National Park.csv
Updated Cargills Square.csv
Updated Casuarina Beach.csv
Updated Colombo City Centre.csv
Updated Colombo Gold Center.csv
Updated Colombo Lotus Tower.csv
Updated Colombo National Museum.csv
Updated Crescat Boulevard Mall.csv
Updated 

In [6]:
# df1 = pd.read_csv(reference_file, encoding="utf-8-sig")
# correct_titles = df1["Destination"].tolist()

# # Keywords to remove
# unwanted_keywords = ["Unclaim", "Someone from this business", "Claim this listing", "If you own this business"]

# for file_name in os.listdir(input_folder):
#     if file_name.endswith(".csv"):
#         file_path = os.path.join(input_folder, file_name)
#         df = pd.read_csv(file_path, encoding="utf-8-sig")
        
#         if "Destination Title" in df.columns:
#             for i, title in enumerate(df["Destination Title"]):
#                 if pd.isna(title):
#                     continue
                
#                 # Remove unwanted keywords
#                 cleaned_title = str(title)
#                 for kw in unwanted_keywords:
#                     cleaned_title = cleaned_title.split(kw)[0].strip()
                
#                 # Fuzzy match to reference
#                 best_match, score = process.extractOne(cleaned_title, correct_titles)
#                 if score >= 80:
#                     df.at[i, "Destination Title"] = best_match
                
#                 # Replace weird characters
#                 df.at[i, "Destination Title"] = df.at[i, "Destination Title"].replace("â", "–").replace("Â", "’")
            
#             # Save CSV with proper UTF-8 encoding
#             df.to_csv(file_path, index=False, encoding="utf-8-sig")
#             print(f"Updated {file_name}")
#         else:
#             print(f"Skipped {file_name}: 'Destination Title' column not found")


In [7]:
def clean_weird_chars(text):
    if pd.isna(text):
        return text
    return (text.replace("â", "’")   
                .replace("â", "–")  
                .replace("Ã¢ÂÂ", "’")
                .replace("Ã¢ÂÂ", "–"))

df["Destination Title"] = df["Destination Title"].apply(clean_weird_chars)

In [5]:
documents = []

for fl in os.listdir(input_folder):
    if not fl.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, fl)
    df = pd.read_csv(file_path)

    if "Review Text" not in df.columns or len(df) == 0:
        continue

    # Destination-level values (same for all rows)
    destination_title = df["Destination Title"].iloc[0]
    destination_score = df["Destination Score"].iloc[0]
    total_reviews = df["Total Reviews (Destination)"].iloc[0]
    destination_image = df["Destination Image"].iloc[0]
    destination_link = df["Destination URL"].iloc[0]

    # Category percentages (already calculated)
    family_pct = df["Family %"].iloc[0]
    couples_pct = df["Couples %"].iloc[0]
    solo_pct = df["Solo %"].iloc[0]
    business_pct = df["Business %"].iloc[0]
    friends_pct = df["Friends %"].iloc[0]
    missing_pct = df["Missing %"].iloc[0]

    # Concatenate all reviews into ONE document
    all_reviews = df["Review Text"].dropna().astype(str)
    document_text = destination_title + ". " + " ".join(all_reviews)

    documents.append({
        "Destination Title": destination_title,"Document Text": document_text,"Destination Score": destination_score,"Total Reviews": total_reviews,
        "Destination Image": destination_image,"Family %": family_pct,"Couples %": couples_pct,"Solo %": solo_pct,"Business %": business_pct,
        "Friends %": friends_pct,"Missing %": missing_pct})
# Create final document CSV
doc_df = pd.DataFrame(documents)
doc_df.to_csv(output_path, index=False)

print(f"Document CSV created successfully with {len(doc_df)} destinations.")

Document CSV created successfully with 53 destinations.


In [9]:
df = pd.read_csv("C:\\NIBM\\Travel recommendation system\\Travel-Recommendation-system-in-Sri-Lanka-\\Data Preparation\\document_data\\documents.csv")

In [10]:
df.head()

,Destination Title,Document Text,Destination Score,Total Reviews,Destination Image,Family %,Couples %,Solo %,Business %,Friends %,Missing %
0,Abayagiriya Stupa,Abayagiriya Stupa. When I visited this place t...,4.3,400,https://dynamic-media-cdn.tripadvisor.com/medi...,0.000000,39.759036,13.654618,2.008032,14.056225,30.522088
1,Aberdeen Falls,"Aberdeen Falls. Welcome to Aberdeen Falls, one...",4.9,33,https://dynamic-media-cdn.tripadvisor.com/medi...,0.000000,12.500000,3.125000,3.125000,34.375000,46.875000
2,Alagalla Mountain Range,Alagalla Mountain Range. A fascinating climb t...,4.7,25,https://dynamic-media-cdn.tripadvisor.com/medi...,0.000000,0.000000,4.545455,0.000000,68.181818,27.272727
3,Ella Rock,Ella Rock. We went there at the middle of our ...,4.3,285,https://dynamic-media-cdn.tripadvisor.com/medi...,21.546961,33.701657,7.734807,1.657459,15.469613,19.889503
4,Ambuluwawa Tower,Ambuluwawa Tower. Ambuluwawa Tower is truly un...,4.6,105,https://dynamic-media-cdn.tripadvisor.com/medi...,0.000000,35.000000,12.500000,2.500000,25.000000,25.000000


In [11]:
df.isna().sum()

Destination Title    0
Document Text        0
Destination Score    0
Total Reviews        0
Destination Image    0
Family %             0
Couples %            0
Solo %               0
Business %           0
Friends %            0
Missing %            0
dtype: int64